In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier,RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier 


from sklearn.datasets import load_breast_cancer, load_wine
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

from warnings import filterwarnings
filterwarnings('ignore')

앙상블 학습의 유형: 보팅(Voting), 배깅(Bagging), 부스팅(Boosting), 스태킹(Stacking) 등

# 1.Voting

- 여러 종류의 알고리즘을 사용한 각각의 결과에 대해 투표를 통해 최종 결과를 예측하는 방식
- 하드 보팅: 다수결의 원칙
- 소프트 보팅: 각 알고리즘이 레이블값 결정 확률을 예측한 결과를 평균내어 확률이 가장 높은 레이블값을 최종 값으로 예측


In [ ]:
cancer = load_breast_cancer()
cancer_df = pd.DataFrame(cancer['data'], columns = cancer['feature_names'])
cancer_df['target'] = cancer['target']

In [ ]:
x_data = cancer['data']
y_data = cancer['target']

x_train, x_test, y_train, y_test = train_test_split(x_data,y_data, test_size = 0.2,
                                                   random_state = 1, stratify = y_data)

In [ ]:
model_logi = make_pipeline(StandardScaler(),LogisticRegression())
model_knn = make_pipeline(StandardScaler(),KNeighborsClassifier())
model_tree = make_pipeline(StandardScaler(),DecisionTreeClassifier())

In [ ]:
# voting = 'hard' or 'soft'
# hard 는 다수결 / soft는 확률
model_vote = VotingClassifier(estimators = [('logi',model_logi),('knn',model_knn),('tree',model_tree)])

model_vote.fit( x_train, y_train)

VotingClassifier(estimators=[('logi', LogisticRegression()),
                             ('knn', KNeighborsClassifier()),
                             ('tree', DecisionTreeClassifier())])

In [ ]:
model_logi.fit( x_train, y_train)
model_logi.predict( x_test)
model_logi.score( x_test, y_test)

0.956140350877193

In [ ]:
model_knn.fit(x_train, y_train)
model_knn.predict( x_test)
model_knn.score( x_test, y_test)

0.956140350877193

In [ ]:
model_vote.predict( x_test )
model_vote.score( x_test, y_test)

0.9649122807017544

In [ ]:
p =model_vote.predict( x_test )
( y_test == p ).mean()

0.9649122807017544

In [ ]:
for c in [model_logi, model_knn, model_tree]:
    c.fit( x_train, y_train)
    print( c.__class__.__name__, c.score( x_test, y_test))

LogisticRegression 0.956140350877193
KNeighborsClassifier 0.956140350877193
DecisionTreeClassifier 0.9473684210526315


## 1-1.연습문제

와인데이터셋에 대해 3개의 분류 클래스를 이용하고 soft voting 으로 정확도를 구하시오.

In [ ]:
from sklearn.datasets import load_wine
wine = load_wine()
x_data = wine['data']
y_data = wine['target']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2,
                                                    stratify=y_data, random_state=1 )

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:
# 로지스틱, knn, decision tree 모델 객체 생성
model_logi = make_pipeline( StandardScaler(),  LogisticRegression() )
model_knn =  make_pipeline( StandardScaler(), KNeighborsClassifier() )
model_tree = make_pipeline( StandardScaler(),  DecisionTreeClassifier() )

In [ ]:
model_vote=VotingClassifier(estimators=[('logi',model_logi),('knn', model_knn),('tree',model_tree)],
                            voting='soft')
model_vote.fit(x_train, y_train)

VotingClassifier(estimators=[('logi',
                              Pipeline(steps=[('standardscaler',
                                               StandardScaler()),
                                              ('logisticregression',
                                               LogisticRegression())])),
                             ('knn',
                              Pipeline(steps=[('standardscaler',
                                               StandardScaler()),
                                              ('kneighborsclassifier',
                                               KNeighborsClassifier())])),
                             ('tree',
                              Pipeline(steps=[('standardscaler',
                                               StandardScaler()),
                                              ('decisiontreeclassifier',
                                               DecisionTreeClassifier())]))],
                 voting='soft')

In [ ]:
model_vote.score( x_test, y_test)

1.0

# 2.Bagging

- bagging은 bootstrap aggregating의 줄임말
- bootstrap:모집단의 성질에 대해 표본을 통해 추정할 수 있는 것처럼, 표본의 성질에 대해서도 재표집(resampling)을 통해 추정할 수 있다는 것이다. 즉 주어진 표본(샘플)에 대해서, 그 샘플에서 또 다시 샘플(재표본)을 여러번(1,000~10,000번, 혹은 그 이상)추출하여 표본의 평균이나 분산 등이 어떤 분포를 가지는가를 알아낼 수 있다.(위키피디아)
- 같은 알고리즘에 대해 데이터 샘플을 다르게 두고 학습을 수행해 보팅을 수행하는 방식
- 이 때의 데이터 샘플은 중첩이 허용된다. 즉 10000개의 데이터에 대해 10개의 알고리즘이 배깅을 사용할 때,각 1000개의 데이터 내에는 중복된 데이터가 존재할 수 있다. 
- 배깅의 대표적인 방식이 Random Forest

In [ ]:
wine = load_wine()

x_data = wine['data']
y_data = wine['target']

x_train, x_test, y_train, y_test = train_test_split(x_data,y_data, test_size = 0.2,
                                                   random_state = 1, stratify = y_data)

In [ ]:
model_knn = make_pipeline(StandardScaler(),KNeighborsClassifier())

In [ ]:
model_bagg = BaggingClassifier(model_knn, n_estimators=10,  # knn모델이 10개가 되는거임
                              max_samples = 0.5, random_state=1)

# knn모델을 10개 만드는데 샘플을 50%로 나눠주는 거임
# 샘플을 랜덤하고 다양하게 나눔

model_bagg.fit(x_train,y_train)

BaggingClassifier(base_estimator=Pipeline(steps=[('standardscaler',
                                                  StandardScaler()),
                                                 ('kneighborsclassifier',
                                                  KNeighborsClassifier())]),
                  max_samples=0.5, random_state=1)

In [ ]:
model_bagg.predict(x_test) # 다수결의 원칙을 사용함

array([1, 1, 2, 0, 2, 0, 2, 1, 2, 2, 0, 1, 1, 2, 1, 1, 1, 0, 0, 0, 1, 0,
       2, 0, 1, 0, 0, 1, 0, 0, 2, 1, 2, 2, 2, 0])

In [ ]:
model_bagg.score(x_test,y_test)

0.9444444444444444

In [ ]:
model_bagg = BaggingClassifier(model_knn, n_estimators=10,
                              max_samples = 0.5, random_state=2)
model_bagg.fit(x_train,y_train)

model_bagg.predict(x_test) 

array([1, 1, 2, 0, 2, 0, 2, 1, 2, 2, 0, 1, 1, 2, 1, 1, 1, 0, 0, 0, 0, 0,
       2, 0, 1, 0, 0, 1, 0, 0, 2, 1, 2, 2, 2, 0])

# 3.RandomForest(Decision Tree  + Bagging)


In [ ]:
forest = RandomForestClassifier()
forest.fit( x_train, y_train)

RandomForestClassifier()

In [ ]:
forest.predict(x_test)

array([1, 1, 2, 0, 2, 0, 2, 1, 2, 1, 0, 1, 1, 2, 1, 1, 1, 0, 0, 0, 1, 0,
       2, 0, 1, 0, 0, 1, 1, 0, 2, 1, 2, 2, 2, 0])

In [ ]:
forest.score(x_test,y_test)

1.0

# 4.Boosting

>여러 개의 알고리즘이 순차적으로 학습을 하되, 앞에 학습한 알고리즘 예측이 틀린 데이터에 대해 올바르게 예측할 수 있도록,    
그 다음번 알고리즘에 가중치(Ada)를 부여하여 학습과 예측을 진행하는 방식   
잔여오차를 다시학습(gradient)   


**부스팅 알고리즘은 대표적으로 아래와 같은 알고리즘들이 있음**
- AdaBoost
- Gradient Booting Machine(GBM)
- XGBoost
- LightGBM
- CatBoost


In [ ]:
model_ada = make_pipeline(StandardScaler(),AdaBoostClassifier()) 
# best_estimator에 원하는 모델 넣어도 됨  default = DecisionTree
# 이 경우에 DecisionTree에다가 50회의 boosting을 준거임

model_ada.fit(x_train,y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('adaboostclassifier', AdaBoostClassifier())])

In [ ]:
model_ada.predict(x_test)

array([1, 1, 2, 0, 2, 1, 2, 1, 2, 1, 0, 1, 1, 2, 1, 1, 1, 0, 0, 0, 0, 0,
       2, 0, 1, 0, 0, 1, 1, 0, 2, 1, 2, 2, 1, 0])

In [ ]:
model_ada.score(x_test,y_test)

0.9166666666666666

In [ ]:
models = [('ada', AdaBoostClassifier()), 
          ('bc', BaggingClassifier()), 
          ('logi',LogisticRegression()), 
          ('knn', KNeighborsClassifier())]
          
model_vote = VotingClassifier(models, voting='soft')
model_vote.fit(x_train,y_train)

VotingClassifier(estimators=[('ada', AdaBoostClassifier()),
                             ('bc', BaggingClassifier()),
                             ('logi', LogisticRegression()),
                             ('knn', KNeighborsClassifier())],
                 voting='soft')

In [ ]:
model_vote.score(x_test,y_test)

0.9444444444444444

## 4-1.xgBoost

Booster Parameters
> eta [default=0.3] => learning_rate
- GBM의 학습 속도와 유사.
- 각 단계에서 가중치를 줄임으로써 모델을 더 강건하게 만든다.
- 일반적으로 0.01-0.2
> min_child_weight [default=1] (Should be tuned using CV)
- child의 관측에서 요구되는 최소 가중치의 합
- over-fitting vs under-fitting을 조정하기 위한 파라미터.
- 너무 큰 값이 주어지면 under-fitting.
> max_depth [default=6] (Should be tuned using CV)
- 트리의 최대 깊이.
- 일반적으로 3-10
> max_leaf_nodes
- 최종 노드의 최대 개수. (max number of terminal nodes)
- 이진 트리가 생성되기 때문에 max_depth가 6이면 max_leaf_nodes는 2^6개가 됨.
> gamma [default=0]
- 분할을 수행하는데 필요한 최소 손실 감소를 지정한다.
- 알고리즘을 보수적으로 만든다. loss function에 따라 조정해야 한다.
> subsample [default=1]
- 각 트리마다의 관측 데이터 샘플링 비율.
- 값을 적게 주면 over-fitting을 방지하지만 값을 너무 작게 주면 under-fitting.
- 일반적으로 0.5-1
> colsample_bytree [default=1]
- 각 트리마다의 feature 샘플링 비율.
- 일반적으로 0.5-1
> lambda [default=1] => reg_lambda
- 가중치에 대한 L2 정규화 용어 (Ridge 회귀 분석과 유사(?))
> alpha [default=0] => reg_alpha
- 가중치에 대한 L1 정규화 용어 (Lasso 회귀 분석과 유사(?))
> scale_pos_weight [default=1]
- 불균형한 경우 더 빠른 수렴(convergence)에 도움되므로 0보다 큰 값을 쓸것.

- binary:logistic : 이진 분류를 위한 로지스틱 회귀, 예측된 확률을 반환한다. (not class)
- multi:softmax : softmax를 사용한 다중 클래스 분류, 예측된 클래스를 반환한다. (not probabilities)
- multi:softprob : softmax와 같지만 각 클래스에 대한 예상 확률을 반환한다.

In [ ]:
from xgboost import XGBClassifier

In [ ]:
xgb = XGBClassifier() # default > tree, boost, 2진분류
# adaboost에서 성능 업그레이드 한 거
xgb.fit(x_train,y_train)

[09:49:33] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=20, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [ ]:
p = xgb.predict(x_test)

In [ ]:
xgb.score(x_test, y_test)

0.9722222222222222